# 0.0 Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import inflection

## 0.1 Helper Functions

In [2]:
def search_outliers(df, series):
    first_quantile = np.quantile(df[series], 0.25)
    third_quantile = np.quantile(df[series], 0.75)
    distance = (third_quantile - first_quantile) * 1.5
    inf_limit = first_quantile - distance
    sup_limit = third_quantile + distance
    index_outliers = df.loc[(df[series]<inf_limit)|(df[series]>sup_limit)].index
    return index_outliers, inf_limit, sup_limit

## 0.2 Loading Data

In [3]:
store = pd.read_csv('D:\\My Drive\\Pessoal\\Projetos\\store_sales_predict\\store.csv', low_memory=False)
train = pd.read_csv('D:\\My Drive\\Pessoal\\Projetos\\store_sales_predict\\train.csv', low_memory=False)
test = pd.read_csv('D:\\My Drive\\Pessoal\\Projetos\\store_sales_predict\\test.csv', low_memory=False)

df = pd.merge(train, store, how='left', on='Store')
df.sample(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
676877,858,6,2013-11-02,4463,388,1,0,0,0,a,a,3370.0,12.0,2008.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct"
134429,630,4,2015-04-02,9504,978,1,1,0,1,a,a,1690.0,4.0,2015.0,0,NaN,NaN,NaN
422856,1057,3,2014-06-18,6177,611,1,1,0,0,d,c,3230.0,11.0,2011.0,0,NaN,NaN,NaN
629733,544,6,2013-12-14,13457,1997,1,0,0,0,a,a,250.0,12.0,2001.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
939012,968,2,2013-03-12,5233,737,1,0,0,0,c,a,1190.0,NaN,NaN,0,NaN,NaN,NaN


# 1.0 Data Description

## 1.1 Rename Columns

In [4]:
# backup df
df1 = df.copy()

snake_case = lambda x: inflection.underscore(x)

cols_old = df1.columns.to_list()
cols_new = list(map(snake_case, cols_old))

# rename
df1.columns = cols_new

## 1.2 Data Dimension

In [5]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

Number of rows: 1017209
Number of columns: 18


## 1.3 Data Types

In [6]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [7]:
# changing data datatype
df1['date'] = pd.to_datetime(df1['date'])

## 1.4 Check NA

In [8]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Fillout NA

In [9]:
# competition_distance
df1.loc[df1['competition_distance'].isna(), 'competition_distance'] = 200000

# competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if np.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1) 

# competition_open_since_year
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if np.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if np.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if np.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

# promo_interval
df1['promo_interval'].fillna(0, inplace=True)
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df1['month_map'] = df1['date'].dt.month.map(month_map)
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

## 1.6 Change Types

In [ ]:
backup = df1.copy()

In [ ]:
df1 = backup.copy()
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype('int')
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype('int')

df1['promo2_since_week'] = df1['promo2_since_week'].astype('int')
df1['promo2_since_year'] = df1['promo2_since_year'].astype('int')

## 1.7 Descriptive Statistical

### 1.7.1 Classifying Attributes

In [ ]:
num_attributes = df1[['sales', 'customers', 'competition_distance']]


cat_attributes = df1[['open', 'state_holiday', 'school_holiday', 'store_type', 'assortment', 'day_of_week', 'promo', 'promo2', 'is_promo']].astype('category')

### 1.7.2 Numerical Attributes

In [ ]:
# Central Tendendy - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T
m.columns = 'min max range mean median std skew kurtosis'.split()
m

### 1.7.3 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.nunique())

In [ ]:
temp = df1.loc[df1['sales'] > 0]

plt.figure(figsize=(20, 20))

plt.subplot(3, 3, 1)
ax = sns.boxplot(x='day_of_week', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('day_of_week', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 2)
ax = sns.boxplot(x='is_promo', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('is_promo', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 3)
ax = sns.boxplot(x='promo', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('promo', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 4)
ax = sns.boxplot(x='promo2', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('promo2', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 5)
ax = sns.boxplot(x='state_holiday', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('state_holiday', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 6)
ax = sns.boxplot(x='school_holiday', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('school_holiday', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 7)
ax = sns.boxplot(x='store_type', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('store_type', fontsize=20)
plt.xlabel('')
plt.ylabel('')

plt.subplot(3, 3, 8)
ax = sns.boxplot(x='assortment', y='sales', data=temp)
ax.tick_params(bottom=False, labelleft=False, left=False)
# ax = sns.despine(left=True)
plt.title('assortment', fontsize=20)
plt.xlabel('')
plt.ylabel('');

# 2.0 Feature Engineering